In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import QuantLib as ql
from scipy.integrate import trapezoid

# Valuation of CMS options - description
This notebook shows implementation of CMS options valuation using the standard market model.

The pricing follows (Brigoa and Mercurio, 2006), Section 13.16.2.

The market model assumes that the inputs are:
- IR curves
- correlation of swap rates underlying the CMS option trade
- volatilities of swap rates underlying the CMS option trade

The option has payoff
\begin{equation}
V({T_\alpha }) = \max \left[ {{S_{\alpha ,\gamma }}({T_\alpha }) - {S_{\alpha ,\beta }}({T_\alpha }) - K,0} \right] \text{where } \gamma > \beta.
\end{equation}

It is convenient to use ${\mathbb{Q}_{{T_\alpha }}}$ measure as it allows us to express the value as
\begin{equation}
V(0) = B(0,{T_\alpha }){\mathbb{E}^{{\mathbb{Q}_{{T_\alpha }}}}}\left[ {\max \left[ {{S_{\alpha ,\gamma }}({T_\alpha }) - {S_{\alpha ,\beta }}({T_\alpha }) - K,0} \right]} \right].
\end{equation}
However, because $S_{\alpha, x}, x \in \{\beta, \gamma\}$ is _not_ a numeraire under ${\mathbb{Q}_{{T_\alpha }}}$ measure, we need to obtain drifts for both ${S_{\alpha ,\gamma }},{S_{\alpha ,\beta }}$ that these processes have under ${\mathbb{Q}_{{T_\alpha }}}$.

Since
\begin{equation}
{R_{{\rm{CMS}}}}(0) = {\mathbb{E}^{{\mathbb{Q}_{{T_\alpha }}}}}\left[ {{S_{\alpha ,x}}({T_\alpha })} \right] = {S_{a,x}}(0){e^{\mu {T_\alpha }}}
\end{equation}
we can back out the drift $\mu$ if we know the CMS rate $R_{\text{CMS}}$ (which is $S_{\alpha, x}(0)+\text{convexity adjustment}$).
We thus need to calucate ${\mathbb{E}^{{\mathbb{Q}_{{T_\alpha }}}}}\left[ {{S_{\alpha ,x}}({T_\alpha })} \right]$ as suggested by (Brigo and Mercurio, 2006) and use approximating formula (13.16) of the book.

TODO: 
- add details about formula for ${\mathbb{E}^{{\mathbb{Q}_{{T_\alpha }}}}}\left[ {{S_{\alpha ,x}}({T_\alpha })} \right]$, 
- add final formula for $V(0)$

In [3]:
today = ql.Date(20, 10, 2022)
ql.Settings.instance().evaluationDate = today

In [4]:
curve = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.NullCalendar(), 0.05, ql.SimpleDayCounter()))

In [5]:
index = ql.Euribor6M(curve)
swap1 = ql.MakeVanillaSwap(ql.Period('5Y'), index, ql.nullDouble(), ql.Period('1Y'), fixedLegTenor = ql.Period('6M'))
swap2 = ql.MakeVanillaSwap(ql.Period('30Y'), index, ql.nullDouble(), ql.Period('1Y'), fixedLegTenor = ql.Period('6M'))

In [6]:
psi = lambda y, curve, day_counter, schedule: sum(day_counter.yearFraction(schedule[i-1], di)/(1+y)**day_counter.yearFraction(schedule[0], di) for i, di in enumerate(schedule[1:], 1))

def get_drift(swap, curve, sigma, h = 0.0001):
    '''
        Drift is the drift of swap rate under T_alpha forward measure
        The drift can be extracted from E^T[S(T)] which requires only knowing swap rate volatility
        
        Below equation (13.16) is implemented.
    '''
    schedule = list(swap.fixedSchedule())
    day_counter = swap.fixedDayCount()
    
    S0 = swap.fairRate()
    
    h = 0.0001
    psi_plus  = psi(S0+h, curve, day_counter, schedule)
    psi_minus = psi(S0-h, curve, day_counter, schedule)
    psi_zero  = psi(S0, curve, day_counter, schedule)

    psi_prime  = (psi_plus - psi_minus)/(2*h)
    psi_second = (psi_plus - 2*psi_zero + psi_minus)/h**2

    Talpha = day_counter.yearFraction(today,schedule[0])
    CMS_rate = S0 - 0.5 * S0**2 *sigma**2 * Talpha * psi_second / psi_prime # implementation of (13.16)
    drift = 1/Talpha * np.log(CMS_rate/S0)
    
    #print(f'S(0) = {S0}, CMS_rate = {CMS_rate}, drift = {drift}')
    return drift


def get_fv(swap1, swap2, curve, v, K, sigma1, sigma2, rho):
    day_counter = swap1.fixedDayCount()

    S1 = swap1.fairRate()
    S2 = swap2.fairRate()
    T = day_counter.yearFraction(today,swap1.fixedSchedule()[0])

    h = lambda K, S, mu, sigma, T, v: K+S*np.exp((mu - 0.5*sigma**2)*T + sigma*np.sqrt(T)*v)
    
    mu1 = get_drift(swap1, curve, sigma1, h = 0.0001)
    mu2 = get_drift(swap2, curve, sigma2, h = 0.0001)
    
    hv = h(K, S1, mu1, sigma1, T, v)
    part1 = S2*np.exp(mu2*T - 0.5*rho**2 * sigma2 **2 *T + rho * sigma2*np.sqrt(T)*v)
    part2 = norm.cdf( (np.log(S2/hv) + (mu2 + (0.5-rho**2)*sigma2**2 )*T + rho*sigma2*np.sqrt(T)*v)/(sigma2*np.sqrt(T)*np.sqrt(1-rho**2))  )
    part3 = hv*norm.cdf( (np.log(S2/hv) + (mu2 - 0.5*sigma2**2)*T + rho*sigma2*np.sqrt(T)*v)/(sigma2*np.sqrt(T)*np.sqrt(1-rho**2)) )

    return part1*part2 - part3

In [7]:
v_grid = np.linspace(-8, 8, 400)
integrand = [1/np.sqrt(2*np.pi) * np.exp(-0.5*v**2) * get_fv(swap1, swap2, curve, v, K = 0.01, sigma1 = 0.2, sigma2 = 0.2, rho = -0.9) for v in v_grid]

In [8]:
undiscounted_payoff = trapezoid(integrand, x=v_grid)
undiscounted_payoff

0.004167010434262846

In [9]:
npv = curve.discount(swap1.fixedSchedule()[0])*undiscounted_payoff
npv

0.003961581447222136